In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)


In [2]:
episodes = pd.read_csv('Datasets/episodes.csv')
paintings = pd.read_csv('Datasets/paintings.csv')
dates = pd.read_fwf('Datasets/dates', header = None)


In [3]:
# Rename the existing column to 'combined'
dates.rename(columns={0: 'combined'}, inplace=True)

# Regex pattern to extract title and date (excluding notes)
pattern = r'^"([^"]+)" \(([^)]+)\)'

# Extract Title and Date  and Notes using regex pattern
dates[['Title', 'Date']] = dates['combined'].str.extract(pattern, expand=True)

# Convert 'Date' to datetime
dates['Date'] = pd.to_datetime(dates['Date'])

# Extract the month and create a new 'Month' column
dates['Month'] = dates['Date'].dt.month_name()

# Drop the 'combined' column
dates.drop(columns=['combined'], inplace=True)


In [4]:
# Convert episode column headers to title case
episodes.columns = episodes.columns.str.title()

# Rename title headers of episodes and paintings files to all match 'Title'
episodes.rename(columns={'TITLE': 'Title'}, inplace=True)
paintings.rename(columns={'painting_title': 'Title'}, inplace=True)

# Convert episode titles in columns 'Title' for each dataset to title case
episodes['Title'] = episodes['Title'].str.title()
dates['Title'] = dates['Title'].str.title()
paintings['Title'] = paintings['Title'].str.title()

# Remove double quotes from the titles in episode DataFrame
episodes['Title'] = episodes['Title'].str.replace('"', '')

# Remove carriage return \r and newline \n characters and single quotes and square brackets from paintings colors column
paintings['colors'] = paintings['colors'].str.replace('\r\n', '').str.replace('[\'\[\]]', '', regex=True)

# Remove single quotes and square brackets from color_hex column in paintings
paintings['color_hex'] = paintings['color_hex'].str.replace('[\'\[\]]', '', regex=True)


In [5]:
# Merge episodes and paintings
merged_df = pd.merge(dates, paintings, on='Title')

# Merge the result with dates DataFrame
final_df = pd.merge(merged_df, episodes, on='Title')

# Drop unnecessary columns season, episode and unnamed
final_df.drop(columns=['season', 'episode', 'Unnamed: 0'], inplace=True)

# Save to new CSV
final_df.to_csv('joyofpainting.csv', index=False)
